In [1]:
import pandas as pd 
import numpy as np 

In [2]:
#reading the 2 files to clean and merge

df1 = pd.read_csv('../data/Crime_Data_from_2010_to_2019.csv') 
df2 = pd.read_csv('../data/Crime_Data_from_2020_to_Present.csv')

In [3]:
#starting the processing of the first file 

df1.columns

Index(['DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA ', 'AREA NAME',
       'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Crm Cd Desc', 'Mocodes',
       'Vict Age', 'Vict Sex', 'Vict Descent', 'Premis Cd', 'Premis Desc',
       'Weapon Used Cd', 'Weapon Desc', 'Status', 'Status Desc', 'Crm Cd 1',
       'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'LOCATION', 'Cross Street', 'LAT',
       'LON'],
      dtype='object')

In [4]:
# Keeping only the columns that I need
# and renaming them

col_list = ['DATE OCC','AREA NAME',
            'Crm Cd Desc','Vict Age', 'Vict Sex', 
            'Vict Descent','Weapon Desc','Premis Desc']      

df1 = df1[col_list]

df1.columns = ['Date','Area','Crime','Vict Age', 'Vict Sex',   
            'Vict Descent','Weapon','Premises']       

In [5]:
df1.head()

,Date,Area,Crime,Vict Age,Vict Sex,Vict Descent,Weapon,Premises
0,02/20/2010 12:00:00 AM,Newton,VIOLATION OF COURT ORDER,48,M,H,NaN,SINGLE FAMILY DWELLING
1,09/12/2010 12:00:00 AM,Pacific,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0,M,W,NaN,STREET
2,08/09/2010 12:00:00 AM,Newton,OTHER MISCELLANEOUS CRIME,0,M,H,NaN,ALLEY
3,01/05/2010 12:00:00 AM,Hollywood,VIOLATION OF COURT ORDER,47,F,W,HAND GUN,STREET
4,01/02/2010 12:00:00 AM,Central,"RAPE, ATTEMPTED",47,F,H,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",ALLEY


In [6]:
df1['Date'] = pd.to_datetime(df1['Date'])   #converting Date to datetime

In [7]:
df1['Month']=df1['Date'].dt.month
df1['Year']=df1['Date'].dt.year
                                                    #creating new columns : Year, Month
year = df1['Year']
month = df1['Month']
df1 = df1.drop(columns=['Date','Year','Month'])
df1.insert(loc=0, column='Month', value=month)
df1.insert(loc=0, column='Year', value=year)

In [8]:
df1.head()

,Year,Month,Area,Crime,Vict Age,Vict Sex,Vict Descent,Weapon,Premises
0,2010,2,Newton,VIOLATION OF COURT ORDER,48,M,H,NaN,SINGLE FAMILY DWELLING
1,2010,9,Pacific,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0,M,W,NaN,STREET
2,2010,8,Newton,OTHER MISCELLANEOUS CRIME,0,M,H,NaN,ALLEY
3,2010,1,Hollywood,VIOLATION OF COURT ORDER,47,F,W,HAND GUN,STREET
4,2010,1,Central,"RAPE, ATTEMPTED",47,F,H,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",ALLEY


In [9]:
df1['Vict Sex'].unique(), df1['Vict Descent'].unique() #unique values of the columns to find all types

(array(['M', 'F', nan, 'X', 'H', '-', 'N'], dtype=object),
 array(['H', 'W', 'B', 'A', 'O', 'X', 'K', nan, 'I', 'J', 'F', 'C', 'P',
        'V', 'U', 'G', 'D', 'S', 'Z', 'L', '-'], dtype=object))

In [10]:
#finding the prevalent Descents and Sexes

df1['Vict Descent'].value_counts() , df1['Vict Sex'].value_counts() 

(H    726809
 W    511027
 B    335644
 O    203264
 X     78258
 A     51181
 K      9183
 F      2575
 C      1071
 I       951
 J       419
 P       346
 V       210
 U       191
 Z       136
 G        85
 S        31
 D        24
 L        18
 -         3
 Name: Vict Descent, dtype: int64,
 M    975520
 F    890637
 X     55224
 H        73
 N        17
 -         1
 Name: Vict Sex, dtype: int64)

In [11]:
#changing types of Descent and Sex to a more suitable way of reading them
#also grouping as other the non-prevalent ones

df1['Vict Sex'].mask(((df1['Vict Sex'] != 'M') & (df1['Vict Sex'] != 'F' )), 'Other', inplace=True)

df1['Vict Sex'] = df1['Vict Sex'].replace(['M', 'F'],
                                        ['Male','Female'])

df1['Vict Descent'].mask(((df1['Vict Descent'] != 'W') & (df1['Vict Descent'] != 'B') & (df1['Vict Descent'] != 'H')
                            & (df1['Vict Descent'] != 'A') & (df1['Vict Descent'] != 'O') & (df1['Vict Descent'] != 'X')),
                        'Other', inplace=True)

df1['Vict Descent'] = df1['Vict Descent'].replace(['H','W','B','O','X','A'],
                                        ['Hispanic','White','Black','Unknown','Unknown','Asian'])

In [12]:
df1.head()

,Year,Month,Area,Crime,Vict Age,Vict Sex,Vict Descent,Weapon,Premises
0,2010,2,Newton,VIOLATION OF COURT ORDER,48,Male,Hispanic,NaN,SINGLE FAMILY DWELLING
1,2010,9,Pacific,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0,Male,White,NaN,STREET
2,2010,8,Newton,OTHER MISCELLANEOUS CRIME,0,Male,Hispanic,NaN,ALLEY
3,2010,1,Hollywood,VIOLATION OF COURT ORDER,47,Female,White,HAND GUN,STREET
4,2010,1,Central,"RAPE, ATTEMPTED",47,Female,Hispanic,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",ALLEY


In [13]:
#starting the processing of the first file 

df2.columns

Index(['DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA', 'AREA NAME',
       'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Crm Cd Desc', 'Mocodes',
       'Vict Age', 'Vict Sex', 'Vict Descent', 'Premis Cd', 'Premis Desc',
       'Weapon Used Cd', 'Weapon Desc', 'Status', 'Status Desc', 'Crm Cd 1',
       'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'LOCATION', 'Cross Street', 'LAT',
       'LON'],
      dtype='object')

In [14]:
# the same process for 2nd file is adapted since the files are identical 
# just contains information about later dates

col_list = ['DATE OCC','AREA NAME',
            'Crm Cd Desc','Vict Age', 'Vict Sex', 
            'Vict Descent','Weapon Desc','Premis Desc']      

df2 = df2[col_list]

df2.columns = ['Date','Area','Crime','Vict Age', 'Vict Sex',   
            'Vict Descent','Weapon','Premises']       

In [15]:
df2.head()

,Date,Area,Crime,Vict Age,Vict Sex,Vict Descent,Weapon,Premises
0,01/08/2020 12:00:00 AM,Southwest,BATTERY - SIMPLE ASSAULT,36,F,B,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",SINGLE FAMILY DWELLING
1,01/01/2020 12:00:00 AM,Central,BATTERY - SIMPLE ASSAULT,25,M,H,UNKNOWN WEAPON/OTHER WEAPON,SIDEWALK
2,09/16/2020 12:00:00 AM,77th Street,VANDALISM - MISDEAMEANOR ($399 OR UNDER),62,M,B,NaN,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)"
3,01/01/2020 12:00:00 AM,N Hollywood,VANDALISM - MISDEAMEANOR ($399 OR UNDER),76,F,W,NaN,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)"
4,01/01/2020 12:00:00 AM,Mission,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",31,X,X,NaN,BEAUTY SUPPLY STORE


In [16]:
df2['Date'] = pd.to_datetime(df2['Date']) #converting to datetime

df2['Month']=df2['Date'].dt.month
df2['Year']=df2['Date'].dt.year
                                                    #creating new columns : Year, Month
year = df2['Year']
month = df2['Month']
df2 = df2.drop(columns=['Date','Year','Month'])
df2.insert(loc=0, column='Month', value=month)
df2.insert(loc=0, column='Year', value=year)

In [17]:
df2.head()

,Year,Month,Area,Crime,Vict Age,Vict Sex,Vict Descent,Weapon,Premises
0,2020,1,Southwest,BATTERY - SIMPLE ASSAULT,36,F,B,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",SINGLE FAMILY DWELLING
1,2020,1,Central,BATTERY - SIMPLE ASSAULT,25,M,H,UNKNOWN WEAPON/OTHER WEAPON,SIDEWALK
2,2020,9,77th Street,VANDALISM - MISDEAMEANOR ($399 OR UNDER),62,M,B,NaN,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)"
3,2020,1,N Hollywood,VANDALISM - MISDEAMEANOR ($399 OR UNDER),76,F,W,NaN,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)"
4,2020,1,Mission,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",31,X,X,NaN,BEAUTY SUPPLY STORE


In [18]:
df2['Vict Sex'].unique(), df2['Vict Descent'].unique()

(array(['F', 'M', 'X', nan, 'H'], dtype=object),
 array(['B', 'H', 'W', 'X', 'A', 'O', nan, 'C', 'F', 'K', 'I', 'V', 'J',
        'G', 'P', 'U', 'S', 'Z', 'D', 'L'], dtype=object))

In [19]:
df2['Vict Descent'].value_counts() , df2['Vict Sex'].value_counts() 

(H    100402
 W     69227
 B     46942
 X     29788
 O     26049
 A      6974
 K      1506
 F       990
 C       799
 J       329
 V       237
 I       203
 Z        87
 P        63
 U        46
 G        25
 D        15
 S        14
 L        11
 Name: Vict Descent, dtype: int64,
 M    138977
 F    117920
 X     26782
 H        34
 Name: Vict Sex, dtype: int64)

In [20]:
df2['Vict Sex'].mask(((df2['Vict Sex'] != 'M') & (df2['Vict Sex'] != 'F' )), 'Other', inplace=True)

df2['Vict Sex'] = df2['Vict Sex'].replace(['M', 'F'],
                                        ['Male','Female'])

df2['Vict Descent'].mask(((df2['Vict Descent'] != 'W') & (df2['Vict Descent'] != 'B') & (df2['Vict Descent'] != 'H')
                            & (df2['Vict Descent'] != 'A') & (df2['Vict Descent'] != 'O') & (df2['Vict Descent'] != 'X')),
                        'Other', inplace=True)

df2['Vict Descent'] = df2['Vict Descent'].replace(['H','W','B','O','X','A'],
                                        ['Hispanic','White','Black','Unknown','Unknown','Asian'])

In [21]:
df2.head()

,Year,Month,Area,Crime,Vict Age,Vict Sex,Vict Descent,Weapon,Premises
0,2020,1,Southwest,BATTERY - SIMPLE ASSAULT,36,Female,Black,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",SINGLE FAMILY DWELLING
1,2020,1,Central,BATTERY - SIMPLE ASSAULT,25,Male,Hispanic,UNKNOWN WEAPON/OTHER WEAPON,SIDEWALK
2,2020,9,77th Street,VANDALISM - MISDEAMEANOR ($399 OR UNDER),62,Male,Black,NaN,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)"
3,2020,1,N Hollywood,VANDALISM - MISDEAMEANOR ($399 OR UNDER),76,Female,White,NaN,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)"
4,2020,1,Mission,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",31,Other,Unknown,NaN,BEAUTY SUPPLY STORE


In [22]:
#now merging the two dataframes 

merged_df = pd.concat([df1,df2] , ignore_index=True)

In [23]:
merged_df = merged_df.sort_values(by=['Year','Month'])


In [24]:
merged_df.head()

,Year,Month,Area,Crime,Vict Age,Vict Sex,Vict Descent,Weapon,Premises
3,2010,1,Hollywood,VIOLATION OF COURT ORDER,47,Female,White,HAND GUN,STREET
4,2010,1,Central,"RAPE, ATTEMPTED",47,Female,Hispanic,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",ALLEY
5,2010,1,Central,SHOPLIFTING - PETTY THEFT ($950 & UNDER),23,Male,Black,NaN,DEPARTMENT STORE
6,2010,1,Central,BURGLARY FROM VEHICLE,46,Male,Hispanic,NaN,STREET
7,2010,1,Central,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",51,Male,Black,UNKNOWN WEAPON/OTHER WEAPON,OTHER PREMISE


In [26]:
merged_df['Vict Age'].unique()

array([ 47,  23,  46,  51,  30,  55,  38,  40,  24,  29,  41,  11,  50,
        18,  37,  28,  25,  31,  36,  61,  43,  35,  32,  26,  27,   0,
        56,  57,  42,  49,  34,  39,  22,  54,  60,  99,  21,  20,  74,
        16,  44,  17,  33,  48,  63,  52,  19,  45,  59,  53,  58,  84,
        68,  82,  62,  66,  65,  64,  14,  69,  13,  15,   4,  12,   9,
         3,   7,  71,   8,  79,  67,  76,   2,  10,   6,  72,  80,  86,
        75,  73,  89,  85,   5,  77,  81,  78,  96,  88,  70,  87,  91,
        90,  92,  83,  97,  95,  98,  94,  93,  -1,  -2,  -3,  -4,  -6,
        -9,  -8, -11, -10,  -5,  -7, 114, 118, 120])

In [ ]:
#a portion of Victim ages are falsely documented so I am going to remove them. 
#I will keep all the people aged from 1 to 99 years old

merged_df = merged_df[((merged_df['Vict Age'] > 0) & (merged_df['Vict Age'] < 100))]

In [30]:
#saving the cleaned file for my analysis

merged_df.to_csv('../data/Crime_Data_from_2010_to_Present(cleaned).csv')